# 10M Parameter Text Generation Model with Google Drive Checkpoints
## Resume Training After Colab Disconnections

This notebook implements automatic checkpoint saving to Google Drive and seamless training resumption.

### 1. Mount Google Drive and Setup

In [ ]:
# Mount Google Drive with error handling
import os
import sys

# Check if running in Colab
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("⚠️ Not running in Google Colab. Using local directory for checkpoints.")

# Mount Google Drive if in Colab
if IN_COLAB:
    try:
        # Check if already mounted
        if os.path.exists('/content/drive/MyDrive'):
            print("✓ Google Drive already mounted")
        else:
            print("Mounting Google Drive...")
            drive.mount('/content/drive', force_remount=False)
            print("✓ Google Drive mounted successfully")
        
        CHECKPOINT_DIR = '/content/drive/MyDrive/text_gen_checkpoints'
        
    except Exception as e:
        print(f"⚠️ Failed to mount Google Drive: {e}")
        print("Using local directory instead (checkpoints will be lost on disconnect)")
        CHECKPOINT_DIR = './checkpoints'
else:
    CHECKPOINT_DIR = './checkpoints'

# Create checkpoint directory
try:
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    print(f"\n📁 Checkpoint directory: {CHECKPOINT_DIR}")
    print(f"✓ Directory ready for saving checkpoints")
except Exception as e:
    print(f"❌ Error creating checkpoint directory: {e}")
    raise

### 2. Install Dependencies

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers datasets tokenizers accelerate wandb evaluate rouge-score nltk sacrebleu
!pip install -q sentencepiece protobuf

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer
from transformers import get_linear_schedule_with_warmup
from datasets import load_dataset
import numpy as np
from tqdm.auto import tqdm
import json
import glob
from datetime import datetime

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

### 3. Checkpoint Management Functions

In [ ]:
def save_checkpoint(model, optimizer, scheduler, epoch, step, train_loss, val_loss, checkpoint_dir, training_history=None):
    """Save checkpoint to Google Drive"""
    checkpoint = {
        'epoch': epoch,
        'step': step,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'training_history': training_history or [],
        'timestamp': datetime.now().isoformat()
    }
    
    checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch{epoch}_step{step}.pt')
    torch.save(checkpoint, checkpoint_path)
    
    # Save latest checkpoint reference
    latest_path = os.path.join(checkpoint_dir, 'latest_checkpoint.txt')
    with open(latest_path, 'w') as f:
        f.write(checkpoint_path)
    
    print(f"✓ Checkpoint saved: {os.path.basename(checkpoint_path)}")
    return checkpoint_path

def load_latest_checkpoint(checkpoint_dir, model, optimizer, scheduler):
    """Load the latest checkpoint from Google Drive"""
    latest_path = os.path.join(checkpoint_dir, 'latest_checkpoint.txt')
    
    if not os.path.exists(latest_path):
        print("No checkpoint found. Starting from scratch.")
        return None
    
    with open(latest_path, 'r') as f:
        checkpoint_path = f.read().strip()
    
    if not os.path.exists(checkpoint_path):
        print("Checkpoint file not found. Starting from scratch.")
        return None
    
    print(f"Loading checkpoint: {os.path.basename(checkpoint_path)}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    print(f"✓ Resumed from epoch {checkpoint['epoch']}, step {checkpoint['step']}")
    print(f"  Previous train loss: {checkpoint['train_loss']:.4f}")
    if checkpoint['val_loss'] > 0:
        print(f"  Previous val loss: {checkpoint['val_loss']:.4f}")
    
    return checkpoint

def cleanup_old_checkpoints(checkpoint_dir, keep_last=3):
    """Keep only the last N checkpoints to save space"""
    checkpoints = sorted(glob.glob(os.path.join(checkpoint_dir, 'checkpoint_*.pt')))
    
    if len(checkpoints) > keep_last:
        for old_checkpoint in checkpoints[:-keep_last]:
            try:
                os.remove(old_checkpoint)
                print(f"  Removed old checkpoint: {os.path.basename(old_checkpoint)}")
            except Exception as e:
                print(f"  Warning: Could not remove {os.path.basename(old_checkpoint)}: {e}")

### 4. Model Configuration

In [ ]:
config = GPT2Config(
    vocab_size=50257,
    n_positions=512,
    n_embd=256,
    n_layer=8,
    n_head=8,
    n_inner=1024,
    activation_function='gelu_new',
    resid_pdrop=0.1,
    embd_pdrop=0.1,
    attn_pdrop=0.1,
)

model = GPT2LMHeadModel(config).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")
print(f"Model size: {total_params * 4 / 1e6:.2f} MB")

### 5. Data Preparation

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

dataset = load_dataset('wikitext', 'wikitext-103-v1')

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=512,
        padding='max_length',
        return_tensors='pt'
    )

tokenized_train = dataset['train'].map(tokenize_function, batched=True, remove_columns=dataset['train'].column_names)
tokenized_val = dataset['validation'].map(tokenize_function, batched=True, remove_columns=dataset['validation'].column_names)

tokenized_train.set_format('torch')
tokenized_val.set_format('torch')

print(f"Train samples: {len(tokenized_train)}")
print(f"Val samples: {len(tokenized_val)}")

### 6. Training Configuration

In [ ]:
# Hyperparameters
BATCH_SIZE = 16
LEARNING_RATE = 5e-4
EPOCHS = 3
WARMUP_STEPS = 500
GRADIENT_ACCUMULATION_STEPS = 4
MAX_GRAD_NORM = 1.0
CHECKPOINT_EVERY_N_STEPS = 500  # Save checkpoint every N steps

train_loader = DataLoader(tokenized_train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(tokenized_val, batch_size=BATCH_SIZE)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
total_steps = len(train_loader) * EPOCHS // GRADIENT_ACCUMULATION_STEPS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps=total_steps)

print(f"Total training steps: {total_steps}")
print(f"Checkpoints will be saved every {CHECKPOINT_EVERY_N_STEPS} steps")

### 7. Check for Existing Checkpoint and Resume

In [ ]:
# Try to load existing checkpoint
checkpoint = load_latest_checkpoint(CHECKPOINT_DIR, model, optimizer, scheduler)

# Set starting point
if checkpoint:
    start_epoch = checkpoint['epoch']
    start_step = checkpoint['step']
    training_history = checkpoint.get('training_history', [])
    print(f"\n🔄 Resuming training from epoch {start_epoch}, step {start_step}")
else:
    start_epoch = 1
    start_step = 0
    training_history = []
    print("\n🆕 Starting fresh training")

### 8. Training Loop with Auto-Checkpointing

In [ ]:
def evaluate(model, loader, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            total_loss += outputs.loss.item()
    
    avg_loss = total_loss / len(loader)
    perplexity = torch.exp(torch.tensor(avg_loss))
    return avg_loss, perplexity.item()

In [ ]:
# Main training loop with checkpoint support
best_val_loss = float('inf')
global_step = start_step

try:
    for epoch in range(start_epoch, EPOCHS + 1):
        print(f"\n{'='*60}")
        print(f"Epoch {epoch}/{EPOCHS}")
        print(f"{'='*60}")
        
        model.train()
        total_loss = 0
        optimizer.zero_grad()
        
        progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch}")
        
        # Calculate starting batch index for resume
        if epoch == start_epoch and start_step > 0:
            # Calculate how many batches to skip based on global_step
            batches_completed = (start_step * GRADIENT_ACCUMULATION_STEPS) % len(train_loader)
            print(f"Resuming from batch {batches_completed}/{len(train_loader)}")
        else:
            batches_completed = 0
        
        for step, batch in enumerate(progress_bar):
            # Skip already completed batches when resuming
            if epoch == start_epoch and step < batches_completed:
                continue
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss / GRADIENT_ACCUMULATION_STEPS
            loss.backward()
            
            if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                global_step += 1
            
            total_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS
            progress_bar.set_postfix({'loss': loss.item() * GRADIENT_ACCUMULATION_STEPS, 'step': global_step})
            
            # Save checkpoint periodically
            if global_step % CHECKPOINT_EVERY_N_STEPS == 0 and global_step > start_step:
                avg_train_loss = total_loss / (step + 1 - batches_completed)
                val_loss, val_perplexity = evaluate(model, val_loader, device)
                
                save_checkpoint(model, optimizer, scheduler, epoch, global_step, avg_train_loss, val_loss, CHECKPOINT_DIR, training_history)
                cleanup_old_checkpoints(CHECKPOINT_DIR, keep_last=3)
                
                model.train()  # Back to training mode
        
        # End of epoch evaluation
        avg_train_loss = total_loss / (len(train_loader) - batches_completed)
        val_loss, val_perplexity = evaluate(model, val_loader, device)
        
        print(f"\nEpoch {epoch} Results:")
        print(f"  Train Loss: {avg_train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        print(f"  Val Perplexity: {val_perplexity:.2f}")
        
        training_history.append({
            'epoch': epoch,
            'train_loss': avg_train_loss,
            'val_loss': val_loss,
            'val_perplexity': val_perplexity,
            'global_step': global_step
        })
        
        # Save end-of-epoch checkpoint
        save_checkpoint(model, optimizer, scheduler, epoch, global_step, avg_train_loss, val_loss, CHECKPOINT_DIR, training_history)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = os.path.join(CHECKPOINT_DIR, 'best_model.pt')
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_loss': val_loss,
                'val_perplexity': val_perplexity
            }, best_model_path)
            print(f"✓ Saved best model (val_loss: {val_loss:.4f})")
        
        cleanup_old_checkpoints(CHECKPOINT_DIR, keep_last=3)
        
        # Reset batches_completed for next epoch
        batches_completed = 0

    print("\n" + "="*60)
    print("✅ Training completed successfully!")
    print(f"Best validation loss: {best_val_loss:.4f}")
    print("="*60)

except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user")
    print("Saving checkpoint before exit...")
    avg_train_loss = total_loss / max(step + 1 - batches_completed, 1)
    save_checkpoint(model, optimizer, scheduler, epoch, global_step, avg_train_loss, 0.0, CHECKPOINT_DIR, training_history)
    print("✓ Checkpoint saved. You can resume training later.")

except Exception as e:
    print(f"\n❌ Error occurred: {e}")
    print("Saving emergency checkpoint...")
    try:
        save_checkpoint(model, optimizer, scheduler, epoch, global_step, 0.0, 0.0, CHECKPOINT_DIR, training_history)
    except:
        print("Could not save emergency checkpoint")
    raise

### 9. Save Training History

In [ ]:
# Save training history to Google Drive
history_path = os.path.join(CHECKPOINT_DIR, 'training_history.json')
with open(history_path, 'w') as f:
    json.dump(training_history, f, indent=2)

print(f"Training history saved to: {history_path}")

# Display training progress
if training_history:
    print("\nTraining Progress:")
    for record in training_history:
        print(f"Epoch {record['epoch']}: Train Loss={record['train_loss']:.4f}, Val Loss={record['val_loss']:.4f}, Perplexity={record['val_perplexity']:.2f}")

### 10. Text Generation

In [ ]:
def generate_text(prompt, max_length=100, temperature=0.8, top_k=50, top_p=0.95):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Test generation
prompts = [
    "The future of artificial intelligence",
    "In a world where technology",
    "Scientists have discovered"
]

print("\n" + "="*60)
print("Text Generation Examples")
print("="*60)

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("-" * 60)
    generated = generate_text(prompt, max_length=150)
    print(generated)
    print()

### 11. Save Final Model to Google Drive

In [ ]:
# Save final model
final_model_dir = os.path.join(CHECKPOINT_DIR, 'final_model')
model.save_pretrained(final_model_dir)
tokenizer.save_pretrained(final_model_dir)

print(f"✓ Final model saved to: {final_model_dir}")
print("\nYour model is safely stored in Google Drive and can be loaded anytime!")

### 12. Load Model from Checkpoint (For Inference)

In [ ]:
# Example: Load the best model for inference
def load_best_model_for_inference():
    best_model_path = os.path.join(CHECKPOINT_DIR, 'best_model.pt')
    
    if os.path.exists(best_model_path):
        checkpoint = torch.load(best_model_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"✓ Loaded best model from epoch {checkpoint['epoch']}")
        print(f"  Val Loss: {checkpoint['val_loss']:.4f}")
        print(f"  Val Perplexity: {checkpoint['val_perplexity']:.2f}")
        return model
    else:
        print("No best model found. Using current model.")
        return model

# Uncomment to load best model
# model = load_best_model_for_inference()

### 13. Checkpoint Management Utilities

In [ ]:
# List all available checkpoints
def list_checkpoints():
    checkpoints = sorted(glob.glob(os.path.join(CHECKPOINT_DIR, 'checkpoint_*.pt')))
    
    if not checkpoints:
        print("No checkpoints found.")
        return
    
    print(f"\nAvailable checkpoints in {CHECKPOINT_DIR}:")
    print("="*60)
    
    for cp in checkpoints:
        try:
            checkpoint = torch.load(cp, map_location='cpu')
            size_mb = os.path.getsize(cp) / (1024 * 1024)
            print(f"\n{os.path.basename(cp)}")
            print(f"  Epoch: {checkpoint['epoch']}, Step: {checkpoint['step']}")
            print(f"  Train Loss: {checkpoint['train_loss']:.4f}, Val Loss: {checkpoint['val_loss']:.4f}")
            print(f"  Size: {size_mb:.2f} MB")
            print(f"  Timestamp: {checkpoint.get('timestamp', 'N/A')}")
        except Exception as e:
            print(f"\n{os.path.basename(cp)} - Error loading: {e}")

# Run to see all checkpoints
list_checkpoints()